In [46]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create scribbles

In [47]:
img,gt = load_food_data(3424)

In [48]:
from scribbles_creator import *

In [52]:
mask = gt==68
sk_prim, sk_sec = double_sk_class(mask)
sk_prim_dil = dilation(sk_prim, square(3))
coords = np.argwhere(sk_prim)
squares = []

def make_square(sk_prim, coords, dil, sq_size):
    sk_prim_dil = dilation(sk_prim, square(dil))
    sq = get_square(sk_prim_dil, coords, sq_size)
    squares.append(sq)
    # from matplotlib import pyplot as plt
    # plt.imshow(sq)
    pix_in_sq = np.sum(sq)
    return pix_in_sq

pix = [make_square(sk_prim, c, 3, 6) for c in coords]
np.min(pix)
len(squares)


190

In [66]:
num_class_pix = [np.sum(gt == class_val) for class_val in set(gt.flatten())]

In [55]:
v = napari.Viewer()
v.add_labels(mask)
v.add_labels(sk_prim, name="prim")
v.add_labels(sk_sec, name = "sec")
v.add_labels(sk_prim_dil, name = "prim dilated")
for sq in squares[0::90]: v.add_labels(sq)

Load the ground truths as batch

In [68]:
img_nums = [3424]#[n for n in range(0, 4900, 856)] #[1382] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 1


Check and filter for resolution

In [58]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
3424:    65,536 pixels


In [59]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 250,000: 1
3424:  65,536 pixels


Define scribbles parameters

In [60]:
# Which scribbles to create
mode = "all"
bins = [0.25]#, 1, 5]
margin = 0.75
scribble_width = 3
# suff = ["TEST"] #["w3"] #standing for width 3
s = "TEST"
rel_scribble_len = 10
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/food_run02"

# What to do
save_res = False
show_res = True
print_steps = True

Loop and create scribbles

In [83]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        np.random.seed(scribbles_seed)
        scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, margin=margin, rel_scribble_len=rel_scribble_len, mode=mode,
                                                        save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps,
                                                        scribble_width=scribble_width)
        percentages[bin].append(perc_labelled)
        print(f"Annotation percentage: {perc_labelled:2.3f}%\n")
        print("\n")

IMG 3424: bin 0.25, suff TEST
CLASS 1:
   sk_max_pix: 20.09, sq_size: 25, sq_pix_range: (20, 150)
--- Sampling squares: pix_in_sk 883 indx_step 1
---    current_coordinate: [73 11] | added_pix (before): 0, pix_in_square: 0, future_total: 0
---    outside sq_pix_range (20, 150)
---    current_coordinate: [154 248] | added_pix (before): 0, pix_in_square: 75, future_total: 75
---    overshoot nr. 0
---    current_coordinate: [152 248] | added_pix (before): 0, pix_in_square: 75, future_total: 75
---    overshoot nr. 1
---    current_coordinate: [  6 148] | added_pix (before): 0, pix_in_square: 59, future_total: 59
---    overshoot nr. 2
---    current_coordinate: [188 241] | added_pix (before): 0, pix_in_square: 135, future_total: 135
---    overshoot nr. 3
---    current_coordinate: [181 244] | added_pix (before): 0, pix_in_square: 127, future_total: 127
---    overshoot nr. 4
---    current_coordinate: [237 240] | added_pix (before): 0, pix_in_square: 93, future_total: 93
---    overshoo

Annotation percentage: 0.221%





Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Check percentages of saved scribbles

In [ ]:
perc_dict = {}
for bin in bins:
    bin_perc_list = []
    for img_num in range(0,540,20):
        scrib = np.array(Image.open(f"{folder_path}{img_num:03d}_scribbles_{mode}_{bin_for_file(bin)}_{s}.png"))
        perc_labelled = np.sum(scrib > 0) / scrib.size
        bin_perc_list.append(perc_labelled)
    perc_dict[bin] = bin_perc_list

Show the last created scribble with its own ground truth, and (optionally) all other ground truths

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")